Based on the work by Savarin (https://github.com/savarin/neural-networks)

In [135]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1)

df = pd.read_csv('./train.csv')

In [155]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout

In [3]:
df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

## Data Preparation

In [247]:
#change datatype of certain object columns to category
for col in ['Sex','Embarked']:
    df[col] = df[col].astype('category')

## data transformation

# encode category to numeric/integer
df['SexCode'] = df['Sex'].cat.codes
df['EmbarkedCode'] = df['Embarked'].cat.codes

# impute NaN with mean
df[['Age']] = df[['Age']].replace( np.NaN, df['Age'].mean())

In [248]:
df_train = df.iloc[:712, :]

In [249]:
df_train.dtypes

PassengerId        int64
Survived           int64
Pclass             int64
Name              object
Sex             category
Age              float64
SibSp              int64
Parch              int64
Ticket            object
Fare             float64
Cabin             object
Embarked        category
SexCode             int8
EmbarkedCode        int8
dtype: object

## Split dataset to train/valid

In [250]:
#df_train = df.iloc[:712, :]

scaler = StandardScaler()
#features = ['Pclass','SexCode','Age','Fare']
features = ['Pclass','SexCode','Age','Fare','SibSp','EmbarkedCode','Parch']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [252]:
df_valid = df.iloc[712:, :]

X_test = scaler.transform(df_valid[features].values)
y_test = df_valid['Survived'].values

## 2-layer NN - v4

In [253]:
start = time()

model = Sequential()
model.add(Dense(input_dim=7, output_dim=100))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim=100))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim=100))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim=2))
model.add(Activation("sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics= ['accuracy'])

model.fit(X_train, y_train_onehot, validation_split=0.33,epochs=300, batch_size=20)

print ('\nTime taken %s seconds' % str(time() - start) )

/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=7, units=100)`
  after removing the cwd from sys.path.
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100)`
  import sys
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100)`
  # Remove the CWD from sys.path while we load stuff.
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2)`
  del sys.path[0]


Train on 477 samples, validate on 235 samples
Epoch 1/300
477/477 [==============================] - 0s - loss: 0.5695 - acc: 0.7453 - val_loss: 0.5032 - val_acc: 0.7702
Epoch 2/300
477/477 [==============================] - 0s - loss: 0.4774 - acc: 0.8092 - val_loss: 0.4893 - val_acc: 0.7681
Epoch 3/300
477/477 [==============================] - 0s - loss: 0.4634 - acc: 0.8092 - val_loss: 0.4850 - val_acc: 0.7745
Epoch 4/300
477/477 [==============================] - 0s - loss: 0.4490 - acc: 0.8155 - val_loss: 0.4813 - val_acc: 0.7660
Epoch 5/300
477/477 [==============================] - 0s - loss: 0.4376 - acc: 0.8187 - val_loss: 0.4963 - val_acc: 0.7745
Epoch 6/300
477/477 [==============================] - 0s - loss: 0.4310 - acc: 0.8208 - val_loss: 0.4982 - val_acc: 0.7851
Epoch 7/300
477/477 [==============================] - 0s - loss: 0.4428 - acc: 0.8229 - val_loss: 0.4932 - val_acc: 0.7787
Epoch 8/300
477/477 [==============================] - 0s - loss: 0.4393 - acc: 0.8197

477/477 [==============================] - 0s - loss: 0.3682 - acc: 0.8595 - val_loss: 0.5099 - val_acc: 0.7872
Epoch 67/300
477/477 [==============================] - 0s - loss: 0.3645 - acc: 0.8438 - val_loss: 0.5162 - val_acc: 0.7809
Epoch 68/300
477/477 [==============================] - 0s - loss: 0.3880 - acc: 0.8501 - val_loss: 0.4801 - val_acc: 0.7957
Epoch 69/300
477/477 [==============================] - 0s - loss: 0.3749 - acc: 0.8386 - val_loss: 0.5101 - val_acc: 0.7872
Epoch 70/300
477/477 [==============================] - 0s - loss: 0.3594 - acc: 0.8459 - val_loss: 0.5161 - val_acc: 0.7894
Epoch 71/300
477/477 [==============================] - 0s - loss: 0.3806 - acc: 0.8407 - val_loss: 0.5116 - val_acc: 0.7872
Epoch 72/300
477/477 [==============================] - 0s - loss: 0.3741 - acc: 0.8428 - val_loss: 0.5232 - val_acc: 0.7723
Epoch 73/300
477/477 [==============================] - 0s - loss: 0.3761 - acc: 0.8386 - val_loss: 0.5493 - val_acc: 0.7702
Epoch 74/300


477/477 [==============================] - 0s - loss: 0.3192 - acc: 0.8700 - val_loss: 0.6154 - val_acc: 0.7723
Epoch 132/300
477/477 [==============================] - 0s - loss: 0.3372 - acc: 0.8606 - val_loss: 0.5840 - val_acc: 0.7851
Epoch 133/300
477/477 [==============================] - 0s - loss: 0.3240 - acc: 0.8669 - val_loss: 0.6085 - val_acc: 0.7702
Epoch 134/300
477/477 [==============================] - 0s - loss: 0.3261 - acc: 0.8627 - val_loss: 0.6674 - val_acc: 0.7660
Epoch 135/300
477/477 [==============================] - 0s - loss: 0.3079 - acc: 0.8763 - val_loss: 0.6370 - val_acc: 0.7872
Epoch 136/300
477/477 [==============================] - 0s - loss: 0.3311 - acc: 0.8574 - val_loss: 0.6225 - val_acc: 0.7936
Epoch 137/300
477/477 [==============================] - 0s - loss: 0.3383 - acc: 0.8595 - val_loss: 0.6468 - val_acc: 0.7872
Epoch 138/300
477/477 [==============================] - 0s - loss: 0.3209 - acc: 0.8637 - val_loss: 0.6688 - val_acc: 0.7702
Epoch 

477/477 [==============================] - 0s - loss: 0.3163 - acc: 0.8616 - val_loss: 0.6741 - val_acc: 0.7979
Epoch 197/300
477/477 [==============================] - 0s - loss: 0.3101 - acc: 0.8637 - val_loss: 0.7197 - val_acc: 0.7766
Epoch 198/300
477/477 [==============================] - 0s - loss: 0.3093 - acc: 0.8721 - val_loss: 0.7124 - val_acc: 0.7894
Epoch 199/300
477/477 [==============================] - 0s - loss: 0.3493 - acc: 0.8679 - val_loss: 0.7272 - val_acc: 0.7851
Epoch 200/300
477/477 [==============================] - 0s - loss: 0.3241 - acc: 0.8742 - val_loss: 0.7640 - val_acc: 0.7915
Epoch 201/300
477/477 [==============================] - 0s - loss: 0.3015 - acc: 0.8711 - val_loss: 0.7254 - val_acc: 0.7915
Epoch 202/300
477/477 [==============================] - 0s - loss: 0.3164 - acc: 0.8532 - val_loss: 0.7038 - val_acc: 0.7787
Epoch 203/300
477/477 [==============================] - 0s - loss: 0.3007 - acc: 0.8690 - val_loss: 0.8145 - val_acc: 0.7766
Epoch 

477/477 [==============================] - 0s - loss: 0.2837 - acc: 0.8700 - val_loss: 0.8278 - val_acc: 0.8064
Epoch 262/300
477/477 [==============================] - 0s - loss: 0.2929 - acc: 0.8753 - val_loss: 0.8031 - val_acc: 0.8000
Epoch 263/300
477/477 [==============================] - 0s - loss: 0.3084 - acc: 0.8742 - val_loss: 0.8120 - val_acc: 0.8085
Epoch 264/300
477/477 [==============================] - 0s - loss: 0.2831 - acc: 0.8816 - val_loss: 0.7682 - val_acc: 0.8085
Epoch 265/300
477/477 [==============================] - 0s - loss: 0.2989 - acc: 0.8857 - val_loss: 0.9154 - val_acc: 0.8064
Epoch 266/300
477/477 [==============================] - 0s - loss: 0.3432 - acc: 0.8690 - val_loss: 0.8167 - val_acc: 0.8000
Epoch 267/300
477/477 [==============================] - 0s - loss: 0.3174 - acc: 0.8679 - val_loss: 0.7866 - val_acc: 0.8021
Epoch 268/300
477/477 [==============================] - 0s - loss: 0.3294 - acc: 0.8616 - val_loss: 0.8483 - val_acc: 0.7979
Epoch 

In [254]:
y_prediction = model.predict_classes(X_test)
print ("\nAccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

 32/179 [====>.........................] - ETA: 2s
Accuracy 0.821229050279


## Prepare submission

PassengerId | Survived
-----------| ------------
892 | 0
893 | 1

In [255]:
df_test = pd.read_csv('./test.csv')

In [256]:
for col in ['Sex','Embarked']:
    df_test[col] = df[col].astype('category')

## data transformation

# encode category to numeric/integer
df_test['SexCode'] = df_test['Sex'].cat.codes
df_test['EmbarkedCode'] = df_test['Embarked'].cat.codes

# impute NaN with mean
df_test[['Age']] = df_test[['Age']].replace( np.NaN, df_test['Age'].mean())
df_test[['Fare']] = df_test[['Fare']].replace( np.NaN, df_test['Fare'].mean())

In [263]:
features = ['Pclass','SexCode','Age','Fare','SibSp','EmbarkedCode','Parch'] # 7 features
#features = ['Pclass','SexCode','Age','Fare'] # 4 features

X_submit = scaler.transform(df_test[features].values)

In [264]:
y_submit = model.predict_classes(X_submit)


#for random forest
#y_submit = model.predict(X_submit)
#print ("\nAccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

 32/418 [=>............................] - ETA: 0s

In [265]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": y_submit
    })

In [266]:
submission.to_csv('submission.csv', index=False)